## RNN model 

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from sklearn.metrics import mean_squared_error, root_mean_squared_error
import tensorflow as tf
import keras_tuner as kt
import pickle
from tensorflow.keras.models import load_model

In [4]:
# Data Import 

# Load X_train
with open("data/X_train.pkl", "rb") as f:
    X_train = pickle.load(f)

# Load y_train
with open("data/y_train.pkl", "rb") as f:
    y_train = pickle.load(f)

# Load X_val
with open("data/X_val.pkl", "rb") as f:
    X_val = pickle.load(f)

# Load y_val
with open("data/y_val.pkl", "rb") as f:
    y_val = pickle.load(f)

# Load X_test
with open("data/X_test.pkl", "rb") as f:
    X_test = pickle.load(f)

# Load y_test
with open("data/y_test.pkl", "rb") as f:
    y_test = pickle.load(f)

In [18]:
def plot_model_rmse_and_loss(history):
    
    # Evaluate train and validation accuracies and losses
    
    #train_rmse = history.history['root_mean_squared_error']
    #val_rmse = history.history['val_root_mean_squared_error']
    
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    # Visualize epochs vs. train and validation accuracies and losses
    
    #plt.figure(figsize=(20, 10))
    #plt.subplot(1, 2, 1)
    #plt.plot(train_rmse, label='Training RMSE')
    #plt.plot(val_rmse, label='Validation RMSE')
    #plt.legend()
    #plt.title('Epochs vs. Training and Validation RMSE')
    
    plt.subplot(1, 2, 2)
    plt.plot(train_loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend()
    plt.title('Epochs vs. Training and Validation Loss')
    
    plt.show()

In [10]:
rnn_model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, input_shape=(None, X_train.shape[2])),
    Dropout(0.1)
    tf.keras.layers.Dense(1)
])

lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 10))
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint("rnn_1.h5", monitor= "val_loss", save_best_only=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=6e-3, amsgrad=True)

rnn_model.compile(optimizer=optimizer, loss='mse')
history = rnn_model.fit(X_train, y_train, epochs=50, validation_data = (X_val, y_val), batch_size=32, verbose=1, callbacks = [early_stopping, model_checkpoint])


Epoch 1/50
845/845 [==============================] - 29s 31ms/step - loss: 0.0084 - val_loss: 0.0014
Epoch 2/50
845/845 [==============================] - 50s 59ms/step - loss: 0.0020 - val_loss: 9.1024e-04
Epoch 3/50
845/845 [==============================] - 37s 43ms/step - loss: 0.0013 - val_loss: 8.4322e-04
Epoch 4/50
845/845 [==============================] - 27s 32ms/step - loss: 0.0011 - val_loss: 8.4327e-04
Epoch 5/50
845/845 [==============================] - 46s 55ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 6/50
845/845 [==============================] - 34s 40ms/step - loss: 9.9788e-04 - val_loss: 9.0027e-04
Epoch 7/50
845/845 [==============================] - 28s 33ms/step - loss: 0.0010 - val_loss: 0.0010
Epoch 8/50
845/845 [==============================] - 23s 27ms/step - loss: 9.9451e-04 - val_loss: 0.0012
Epoch 9/50
845/845 [==============================] - 41s 48ms/step - loss: 9.8298e-04 - val_loss: 6.4217e-04
Epoch 10/50
845/845 [=============================

In [17]:
rnn_model = load_model("rnn_1.h5")
predictions = rnn_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'RMSE: {rmse:.4f}')

125/125 [==============================] - 2s 13ms/step
RMSE: 0.0204


In [1]:
plot_model_rmse_and_loss(history)

NameError: name 'plot_model_rmse_and_loss' is not defined

In [7]:
print(X_train.shape)

(27024, 24, 17)


In [11]:
# Funktion zum Erstellen des Modells mit Hyperparametern
def build_rnn_model(hp):
    model = tf.keras.Sequential()
    
    # RNN-Schicht mit variabler Anzahl an Neuronen
    model.add(SimpleRNN(
        units=hp.Int('rnn_units', min_value=16, max_value=128, step=16), 
        input_shape=(None, X_train.shape[2])
    ))
    
    # Dense-Schicht mit variabler Neuronenzahl
    model.add(Dense(
        units=hp.Int('dense_units', min_value=8, max_value=64, step=8),
        activation='relu'
    ))
    
    # Output-Schicht
    model.add(Dense(1))
    
    # Optimizer mit variabler Lernrate
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    )
    
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Keras Tuner: Grid Search als Beispiel
tuner = kt.Hyperband(
    build_rnn_model,  # Modellfunktion
    objective='val_loss',  # Ziel: Validation Loss minimieren
    max_epochs=10,  # Maximale Anzahl an Epochen pro Trial
    factor=3,  # Anzahl der Trials wird mit jedem Schritt um den Faktor erhöht
    hyperband_iterations=3,  # Die Anzahl der Hyperband-Iterationen
    directory='keras_tuner_rnn',  # Speicherort
    project_name='rnn_hyperband',
    max_parallel_trials = 5
)

# Hyperparameter-Tuning ausführen
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Bestes Modell abrufen
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Beste RNN-Einheiten: {best_hps.get('rnn_units')}")
print(f"Beste Dense-Einheiten: {best_hps.get('dense_units')}")
print(f"Beste Lernrate: {best_hps.get('learning_rate')}")

# Bestes Modell neu trainieren
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint("rnn_best.h5", monitor= "val_loss", save_best_only=True)
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), verbose = 1, callbacks = [early_stopping, model_checkpoint])



ValueError: Unrecognized arguments ['max_parallel_trials'] for `BaseTuner.__init__()`.

In [4]:
import os
os.cpu_count() 

22